# PREVISIÓN MODELO CHURN

## Librerías

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

## Carga de modelos, variables y transformaciones

In [2]:
pipeline = pickle.load(open('../pipelines/pipeline.pkl', 'rb')) # scaler
modelo = pickle.load(open('../modelos/model.pkl', 'rb')) # modelo entrenado
var_significant =pd.read_csv("../input_vars/var_significant.csv") # variables a utilizar

# 1. Tablón de previsión

In [3]:
# Cargamos variables input a fecha actual

clientes_df = pd.read_csv("../datos/clientes_2024-01-01.csv", sep='|')
consumos_df = pd.read_csv("../datos/consumos_2024-01-01.csv", sep='|')
financiacion_df = pd.read_csv("../datos/financiacion_2024-01-01.csv",sep='|')
productos_df = pd.read_csv("../datos/productos_2024-01-01.csv", sep= '|')

In [4]:
# Unimos variables input en un dataframe

df = clientes_df.merge(consumos_df, on="id", how="left")
df = df.merge(financiacion_df, on="id", how="left")
df = df.merge(productos_df, on="id", how="left")

Se realiza exactamente el mismo preprocesamiento que el realizado sobre el tablon de train:

In [5]:
df2 = df.copy() # creamos una copia para no sobrescribir el df original

In [6]:
df2.loc[df2["vel_conexion"] < 0, 'vel_conexion'] = np.nan
df2.loc[df2['conexion']=='adsl', 'conexion'] = 'ADSL'

In [7]:
df2 = df2.fillna({'descuentos':'NO', 'financiacion':'NO', 'incidencia':'NO'})
df2 = df2.fillna({'num_dt':0, 'imp_financ':0})
df2 = df2.fillna({'vel_conexion':df2['vel_conexion'].mean()})

In [8]:
df2['financiacion'] = np.where(df2['financiacion']=="SI", 1, 0)
df2['incidencia'] = np.where(df2['incidencia']=="SI", 1, 0)
df2['descuentos'] = np.where(df2['descuentos']=="SI", 1, 0)

In [9]:
año_actual = datetime.now().year

In [10]:
df2['antiguedad'] = año_actual - df2['antiguedad']

# 2. Selección variables y transformadores

In [11]:
# Seleccionamos variables cargadas

variables = var_significant['variables'].tolist()

X = df2[variables]

In [12]:
# Aplicamos transformador cargado

X_fin = pipeline.transform(X)

/Applications/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# 3. Obtenemos predicciones

In [13]:
# Aplicamos modelo cargado

predicciones = modelo.predict_proba(X_fin)

In [14]:
# Debemos tener la prediccion asociada a cada idcliente:

df['probabilidad'] = predicciones[:,1]
df_pred = df[['id', 'probabilidad']]

In [15]:
df_pred['decil'] = pd.qcut(df_pred['probabilidad'], 10, labels = range(1,11))

/var/folders/ms/b0wp19xj6t95pygkb1yqjh2c0000gn/T/ipykernel_1791/1090406611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['decil'] = pd.qcut(df_pred['probabilidad'], 10, labels = range(1,11))


In [16]:
df_pred.to_csv('../resultados/predicciones.csv', index = False)

In [17]:
df_pred.head()

,id,probabilidad,decil
0,1,0.071931,8
1,2,0.002105,2
2,3,0.005447,3
3,4,0.027109,6
4,6,0.065244,8
